# Вступительное задание, направление Глубокое Обучение

В рамках этого задания нужно:
- установить основные библиотеки для глубокого обучения
- скачать датасет CIFAR10
- собрать свою собственную нейронную сеть 
- обучить эту сеть на CIFAR10 и измерить качество на тестовой выборке

## Настройка репозитория

Для того, чтобы успешно выполнить это задание, а также подготовиться к выполнению семинарских и домашних заданий нужно настроить [репозиторий курса](https://github.com/data-mining-in-action/DMIA_DL_2019_Spring). В README.md репозитория подробно описано, как это сделать.

### Задание 1
Чтобы удостовериться, что пакеты установлены правильно нужно выполнить следующий код:
```
out_pip_list = !pip list
for package_info in out_pip_list:
    if package_info.startswith('tensorflow'):
        print(package_info)
    if package_info.startswith('Keras '):
        print(package_info)
```

Версии пакетов в выводе кода будут ответом на пункт 1 этого задания. Их нужно записать в форму в формате: 
```
keras:n.n.n,tensorflow:n.nn.n
```

## Скачивание CIFAR10

### Задание 2
Существует множество способов скачать [CIFAR10](https://en.wikipedia.org/wiki/CIFAR-10), но мы воспользуемся, вероятно, самым простым из них, а именно воспользуемся функциональностью библиотеки [keras](https://keras.io) (которую только что установили). Чтобы все сделать правильно, нужно выполнить следующие действия:
- воспользоваться функцией load_data модуля keras.datasets.cifar10, можно использовать следующие названия переменных: (x_train, y_train), (x_test, y_test)
- выполнить команду ```x_train.shape[0] + y_train.shape[0]```, вывод которой будет ответом на пункт 2 этого задания

In [ ]:
import keras
import matplotlib.pyplot as plt
import numpy as np

from numpy.random import seed
from tensorflow import set_random_seed

In [ ]:
seed(1)
set_random_seed(2)

In [ ]:
(x_train, y_train), (x_test, y_test) = #код для загрузки CIFAR10

Посмотрим на картинки из CIFAR10.

In [ ]:
class_names = {0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer',
               5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}

In [ ]:
fig = plt.figure(figsize=(8,3))

for i in range(len(class_names)):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    ax.set_title(class_names[i])
    idx = np.where(y_train[:]==i)[0][0]
    image = x_train[idx]
    plt.imshow(image)

plt.show()

## Создание модели в keras

Из всего множества архитектур сверточных нейронных сетей мы соберем одну очень простую и прямолинейную. Чтобы выполнить задание, нужно вписать код в специально отведенные места. Получившийся ответ нужно вписать в форму ответа на пункт 3 этого задания.

In [ ]:
from IPython.display import SVG
from sklearn.metrics import accuracy_score
from keras.utils.vis_utils import model_to_dot
from keras.models import Sequential
from keras.layers import (Dense, Conv2D, Activation,
                          Dropout, MaxPooling2D, Flatten)
from keras.optimizers import Adam
from keras.utils import to_categorical

В клетке ниже нужно создать объект класса Sequential из модуля keras.models и добавить в него блоки, тем самым собрав сверточную нейронную сеть, который мы будем классифицировать картинки из CIFAR10.

Сеть должна иметь следующую структуру:
```
- Conv2D(filters=32, kernel_size=(3, 3), padding='same', input_shape=x_train.shape[1:]))
- Activation(activation='relu')
- Conv2D(filters=32, kernel_size=(3, 3), padding='same')
- Activation(activation='relu')
- MaxPooling2D(pool_size=(2, 2))
- Dropout(rate=0.25)
- Conv2D(filters=64, kernel_size=(3, 3), padding='same')
- Activation(activation='relu')
- Conv2D(filters=64, kernel_size=(3, 3), padding='same')
- Activation(activation='relu')
- MaxPooling2D(pool_size=(2, 2))
- Dropout(rate=0.25)
- Flatten()
- Dense(units=128)
- Activation(activation='relu')
- Dropout(rate=0.5)
- Dense(units=10)
- Activation(activation='softmax', name='out')
```

In [ ]:
def create_model():
    ##########################################
    created_model = # код для создания объекта класса Sequential из модуля keras.models
    ##########################################
    return created_model

def add_blocks_to_model(created_model):
    
    created_model.add(Conv2D(filters=32,
                         kernel_size=(3, 3),
                         padding='same',
                         input_shape=x_train.shape[1:]))
    created_model.add(Activation('relu'))
    
    ##########################################
    # код для добавления блоков из списка выше
    ##########################################
    
    created_model.add(Flatten())
    created_model.add(Dense(units=128))
    created_model.add(Activation('relu'))
    created_model.add(Dropout(0.5))
    created_model.add(Dense(units=10))
    created_model.add(Activation('softmax', name='out'))
    
    return created_model

In [ ]:
model = create_model()

In [ ]:
model = add_blocks_to_model(model)

Посмотрим на количество параметров модели и на размеры входов и выходов слоев.

In [ ]:
model.summary()

Нарисуем модель.

In [ ]:
SVG(model_to_dot(model).create(prog='dot', format='svg'))

### Задание 3
В ответе к заданию 3 нужно привести число, которое получится, если из числа нейронов во Flatten() блоке (это число нужно взять из результата команды `model.summary()` выше) вычесть количество классов, которые мы распознаем в этом задании.

## Обучение модели в keras

Чтобы обучить модель в keras, сначала ее нужно скомпиллировать. Мы скомпиллируем модель следующим образом:
- модель должна использовать оптимизатор, который определен в первой строчке клетки ниже
- функцией потерь для обучения должна быть 'categorical_crossentropy'
- метрикой, которая нас интересует - accuracy

In [ ]:
opt = Adam(lr=0.0001)

##########################################
# код для компиляции модели
##########################################

In [ ]:
history = model.fit(
    x=x_train/255.,
    y=to_categorical(y_train, num_classes=10),
    batch_size=32,
    epochs=10,
    validation_data=(x_test/255., to_categorical(y_test, num_classes=10)),
    shuffle=True
)

In [ ]:
preds_dl_train = model.predict(x_train/255., verbose=1, batch_size=64)
preds_dl_test = model.predict(x_test/255., verbose=1, batch_size=64)

### Задание 4
В ответе к заданию 4 нужно привести число, которое получится, если вывести результат сравнения test accuracy > train accuracy и взять int() от этого результата.

In [ ]:
print('CNN train accuracy')
train_acc = accuracy_score(y_train.flatten(),
                           np.argmax(preds_dl_train, axis=1))
print(train_acc)
print('')
print('CNN test accuracy')
test_acc = accuracy_score(y_test.flatten(),
                          np.argmax(preds_dl_test, axis=1))
print(test_acc)